### Imports

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re
import pandas as pd

### URLs

In [2]:
url_nasa = "https://mars.nasa.gov/news"
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_mars = "https://space-facts.com/mars/"
url_astro = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Browser Setup

In [3]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path,headless=True)

### NASA Articles

In [4]:
browser.visit(url_nasa)
soup = bs(browser.html,'html.parser')

article_titles = soup.find_all(class_='content_title')
article_teasers = soup.find_all(class_='article_teaser_body')

first_teaser = article_teasers[0].text
first_title = article_titles[1].text #using pop(0) was causing problems due to timing of downloading pages.

print("Title: " + first_title + "\nDescription: " + first_teaser)

Title: NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
Description: Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


### JPL Images

In [5]:
browser.visit(url_jpl)
soup = bs(browser.html,'html.parser')

carousel_image = soup.find(class_='carousel_item')
image_location = re.search("background-image: url\(\'(.*)\'\)",carousel_image['style'])

featured_image_url = "https://www.jpl.nasa.gov" + image_location.group(1)
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20063-1920x1200.jpg'

### Mars Facts

In [6]:
table = pd.read_html(url_mars)
table[0].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

### Mars Hemispheres

In [9]:
browser.visit(url_astro)
soup = bs(browser.html,'html.parser')

items = soup.find_all(class_='item')
len(items)

4

In [16]:
hemisphere_image_urls = []

for item in items:
    link = item.a['href']
   
    browser.visit("https://astrogeology.usgs.gov/" + link)
    soup = bs(browser.html,"html.parser")
    
    title = soup.find('h2').text[:-9]

    file_div = soup.find_all('div',class_="downloads")
    file_url = file_div[0].find_all('li')[1].a['href']
    
    hemisphere_image_urls.append({"title":title, "image_url": file_url})
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
